In [5]:
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, models, transforms
import torch
import torch.optim as optim
import torch.nn as nn
import numpy as np
import math
import time
from pathlib import Path


class FoodTriplets(Dataset):
    """
    Class to load food triplets. Individual items consist of:
     - x: concatenated image features of a triplet (ABC)
     - y: corresponding label (1: B is more similar, 0: C is more similar, None: no label)
    """
    def __init__(self, features_file, triplets_file, is_labelled_data=False):
        print("initializing " + str(triplets_file) + " dataset...")
        self.img_features = pd.read_csv("data/" + features_file, header=None, index_col=0)
        self.triplets = pd.read_csv("data/" + triplets_file, sep=" ", header=None).to_numpy()
        self.is_labelled_data = is_labelled_data
        self.labels = None

        if self.is_labelled_data:
            # For each triplet ABC add a triplet ACB
            # Also add corresponding labels
            dim = self.triplets.shape
            extended_triplets = np.zeros((2*dim[0], dim[1]), dtype=np.int32)
            self.labels = np.zeros(2*dim[0], dtype=np.bool)
            idx = 0
            for triplet in self.triplets:
                extended_triplets[idx, :] = triplet
                extended_triplets[idx + 1, :] = [triplet[0], triplet[2], triplet[1]]
                self.labels[idx] = 1  # label at idx+1 is already initialized to 0
                idx += 2

            self.triplets = extended_triplets

        print("done")

    def __len__(self):
        return len(self.triplets)

    def __getitem__(self, idx):
        triplet = self.triplets[idx]
        a, b, c = triplet[0], triplet[1], triplet[2]
        a_features = self.img_features.loc[[a]].to_numpy(dtype=np.float32)
        b_features = self.img_features.loc[[b]].to_numpy(dtype=np.float32)
        c_features = self.img_features.loc[[c]].to_numpy(dtype=np.float32)
        features = np.concatenate((a_features, b_features, c_features), axis=1)
        label = 0  # dummy label
        if self.is_labelled_data:
            label = np.array([[self.labels[idx]]])
        return {"x": features, "y": label}


def get_simple_nn():
    net = nn.Sequential(
        nn.ReLU(inplace=True),
        nn.Dropout(p=0.8, inplace=False),
        nn.Linear(in_features=12288, out_features=6144),
        nn.ReLU(inplace=True),
        nn.Dropout(p=0.8, inplace=False),
        nn.Linear(in_features=6144, out_features=3072),
        nn.ReLU(inplace=True),
        nn.Dropout(p=0.8, inplace=False),
        nn.Linear(in_features=3072, out_features=1500),
        nn.ReLU(inplace=True),
        nn.Dropout(p=0.8, inplace=False),
        nn.Linear(in_features=1500, out_features=500),
        nn.ReLU(inplace=True),
        nn.Dropout(p=0.8, inplace=False),
        nn.Linear(in_features=500, out_features=250),
        nn.ReLU(inplace=True),
        nn.Dropout(p=0.8, inplace=False),
        nn.Linear(in_features=250, out_features=1),
        nn.Sigmoid()
    )
    return net

# def get_first_try_nn():
#     # input 3x1000
#     # L1    1500
#     # L2     750
#     # L3
#     # OUTPUT  1
#     net = nn.Sequential(
#         nn.ReLU(inplace=True),
#         nn.Dropout(p=0.8, inplace=False),
#         nn.Linear(in_features=12288, out_features=6144),
#         nn.ReLU(inplace=True),
#         nn.Dropout(p=0.8, inplace=False),
#         nn.Linear(in_features=6144, out_features=3072),
#         nn.ReLU(inplace=True),
#         nn.Dropout(p=0.8, inplace=False),
#         nn.Linear(in_features=3072, out_features=1500)
#         nn.ReLU(inplace=True),
#         nn.Dropout(p=0.8, inplace=False),
#         nn.Linear(in_features=1500, out_features=500)
#         nn.ReLU(inplace=True),
#         nn.Dropout(p=0.8, inplace=False),
#         nn.Linear(in_features=500, out_features=250)
#         nn.ReLU(inplace=True),
#         nn.Dropout(p=0.8, inplace=False),
#         nn.Linear(in_features=250, out_features=1)
#     )

#     return net

def timer(start, end):
    hours, rem = divmod(end-start, 3600)
    minutes, seconds = divmod(rem, 60)
    return "{:0>2}:{:0>2}:{:05.2f}".format(int(hours), int(minutes), seconds)

 
torch.manual_seed(42)

num_epochs = 2  # each epoch is one pass over the whole dataset
batch_size = 32  # how many samples per training step (32,64,128)
num_workers = 7  # how many CPU cores to use

# Specify some file names
img_features_file = "train_image_features_vgg16bn.csv"
train_triplets_file = "train_triplets.txt" #"train_triplets.txt"
test_triplets_file = "test_triplets.txt" #"test_triplets.txt"

# initialize datasets and dataloaders
train_data = FoodTriplets(img_features_file, train_triplets_file, is_labelled_data=True)
train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=num_workers)
test_data = FoodTriplets(img_features_file, test_triplets_file, is_labelled_data=False)
test_dataloader = DataLoader(test_data, batch_size=None, shuffle=False, num_workers=num_workers)

initializing train_triplets.txt dataset...
done
initializing test_triplets.txt dataset...
done


In [6]:
def train_model(model, num_epochs=30):
    # Set model to training mode
    model.train()

    model.to(device)
    
    loss_function = nn.MSELoss() #nn.BCEWithLogitsLoss()  # combines sigmoid with binary cross entropy loss function
    optimizer = optim.Adam(model.parameters(), lr=0.01)
    
    n_batches = math.ceil(len(train_data) / batch_size)
 
    print(f"training started at {time.ctime()}")
    start_t = time.time()
    print_frequency = 100
    
    for epoch in range(num_epochs):
        running_loss = 0.0
#         print("Epoch {}/{}".format(epoch, num_epochs-1))
 
        for i, data in enumerate(train_dataloader):
            inputs = data["x"].float()
            labels = data["y"].float()
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()   

            output = model(inputs)
            
            loss = loss_function(output, labels)
            loss.backward() # Calculate gradients
            optimizer.step() # Update weights

            # print statistics
            running_loss += loss.item()
            if i % print_frequency == 0:
                loss_history.append(running_loss/print_frequency)
#                 print("%d, %5d, loss: %.3f" % (epoch, i, running_loss/100))
                print(f"Epoch: {epoch+1}/{num_epochs}, Batch: {i}/{n_batches}, loss: {running_loss/print_frequency}, "
                  f"Time elapsed: {timer(start_t, time.time())}")
                running_loss = 0

        print(f"Epoch {epoch} finished")

    print("done")



device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

loss_history = []

model = get_simple_nn()

train_model(model=model,num_epochs=num_epochs)

cuda:0
training started at Sat May 22 15:50:42 2021
Epoch: 1/2, Batch: 0/3720, loss: 0.0025200599431991575, Time elapsed: 00:00:00.52
Epoch: 1/2, Batch: 100/3720, loss: 0.496122305393219, Time elapsed: 00:00:29.49
Epoch: 1/2, Batch: 200/3720, loss: 0.4965625, Time elapsed: 00:00:58.53
Epoch: 1/2, Batch: 300/3720, loss: 0.4965625, Time elapsed: 00:01:27.85


KeyboardInterrupt: 

In [4]:
# model = model.to(device)

# predict on test_data
# model_pred = nn.Sequential(model, nn.Sigmoid())  # add sigmoid to get output in [0,1]
predictions = np.zeros(len(test_data))
print("predicting on test_data...")
l_test = len(test_data)
start_t = time.time()

# Set model to evaluating mode
model.eval()
# model.to(device)
model_pred.eval()
# model_pred.to(device)

for i, data in enumerate(test_dataloader, 0):
    x = data["x"].float()
    
    x = x.to(device)
    predictions[i] = model(x)
    # predictions[i] = model_pred(x)
    if i % 1000 == 0:
        print(f"Predicted: {i}/{l_test}, Time elapsed: {timer(start_t, time.time())}")
        
        
# predictions_INT = pd.DataFrame(predictions).astype(int)
predictions_DF = pd.DataFrame(predictions)
predictions_rounded = predictions_DF.round(0)

# print("saving predictions...")
# predictions_INT.to_csv('data/predictions.csv', index=False, header=False)
print("done")

predicting on test_data...
Predicted: 0/59544, Time elapsed: 00:00:07.51


KeyboardInterrupt: 

In [ ]:
# res = pd.DataFrame(predictions_rounded).astype(int)
res = pd.DataFrame(predictions_rounded)
print("saving predictions...")
res.to_csv('data/predictions_sigmoid.csv', index=False, header=False)
print("done")

res.head(100)

In [ ]:
Path("model").mkdir(parents=True, exist_ok=True)  # create folder if necessary
torch.save(model.state_dict(), "model/params_6_layers")
# load: model.load_state_dict(torch.load("model/params"))
